In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt

In [2]:
# Custom Self-Attention Layer
class SelfAttention(layers.Layer):
    def __init__(self, channels, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)
        self.channels = channels
        self.attention_dense = layers.Dense(channels, activation='sigmoid')

    def call(self, inputs):
        attention_weights = self.attention_dense(inputs)
        return inputs * attention_weights

In [3]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Update the precision and recall for each batch
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        # Calculate F1 score as the harmonic mean of precision and recall
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [4]:
# Data preparation (same as in your code)
train_datagen = ImageDataGenerator(rescale=1./255)
training_set1 = train_datagen.flow_from_directory(
    "Monkeypox/archive (60)/Augmented Images/Augmented Images/FOLDS_AUG/fold1_AUG/Train/",
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',  # Use 'sparse' if your labels are integers (not one-hot encoded)
)

Found 7518 images belonging to 6 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set1 = test_datagen.flow_from_directory(
    "Monkeypox/archive (60)/Augmented Images/Augmented Images/FOLDS_AUG/fold1_AUG/Train/",
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
)

Found 7518 images belonging to 6 classes.


In [6]:
# CNN model with Self-Attention and Bidirectional GRU
cnn = Sequential()

In [7]:
cnn.add(layers.Conv2D(filters=32, padding="same", kernel_size=3, activation='relu', strides=2, input_shape=[64, 64, 3]))
cnn.add(SelfAttention(channels=32))
cnn.add(layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(layers.Conv2D(filters=32, padding='same', kernel_size=3, activation='relu'))
cnn.add(layers.MaxPool2D(pool_size=2, strides=2))

# Ensure the correct reshaping before GRU
cnn.add(layers.Reshape((-1, 32)))  # Reshape for GRU (batch_size, time_steps, features)

# Bidirectional GRU layer
cnn.add(layers.Bidirectional(layers.GRU(64, return_sequences=True)))

cnn.add(layers.Flatten())
cnn.add(layers.Dense(units=128, activation='relu'))
cnn.add(layers.Dense(6, activation='softmax'))

C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Compile the model with Precision, Recall, and custom F1 score
cnn.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy', 'Precision', 'Recall', F1Score()]
)

In [9]:
# Model summary
cnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 32, 32, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ self_attention (SelfAttention)       │ (None, 32, 32, 32)          │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 16, 16, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 8, 8, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 64, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 64, 128)             │          37,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │       1,048,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,098,310 (4.19 MB)

 Trainable params: 1,098,310 (4.19 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Train the model and capture the history
history = cnn.fit(
    training_set1,
    validation_data=test_set1,
    epochs=100
)

Epoch 1/100


C:\Users\KIIT\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


235/235 ━━━━━━━━━━━━━━━━━━━━ 29s 108ms/step - Precision: 0.4201 - Recall: 0.0397 - accuracy: 0.3767 - f1_score: 0.0708 - loss: 1.5927 - val_Precision: 0.7573 - val_Recall: 0.2531 - val_accuracy: 0.5265 - val_f1_score: 0.3794 - val_loss: 1.2579
Epoch 2/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 38s 162ms/step - Precision: 0.6920 - Recall: 0.3318 - accuracy: 0.5454 - f1_score: 0.4475 - loss: 1.2226 - val_Precision: 0.8003 - val_Recall: 0.4473 - val_accuracy: 0.6451 - val_f1_score: 0.5739 - val_loss: 0.9783
Epoch 3/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 45s 192ms/step - Precision: 0.7557 - Recall: 0.5029 - accuracy: 0.6376 - f1_score: 0.6038 - loss: 0.9610 - val_Precision: 0.8552 - val_Recall: 0.5579 - val_accuracy: 0.7211 - val_f1_score: 0.6753 - val_loss: 0.7986
Epoch 4/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 48s 204ms/step - Precision: 0.8327 - Recall: 0.6458 - accuracy: 0.7407 - f1_score: 0.7269 - loss: 0.7239 - val_Precision: 0.8768 - val_Recall: 0.7487 - val_accuracy: 0.8102 - val_f1_score: 0.8077 - val_l

In [11]:
# Save Loss plot
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('loss_plot_bigru.png')  # Save the plot as a PNG file
plt.close()

# Save Accuracy plot
plt.figure()
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title("Accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.savefig('accuracy_plot_bigru.png')  # Save the plot as a PNG file
plt.close()

# Save Precision plot (Correct key for precision)
plt.figure()
plt.plot(history.history['Precision'], label='Train Precision')  # Corrected for precision
plt.plot(history.history['val_Precision'], label='Validation Precision')  # Corrected for validation precision
plt.legend()
plt.title("Precision")
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.savefig('precision_plot_bigru.png')  # Save the plot as a PNG file
plt.close()

# Save Recall plot (Correct key for recall)
plt.figure()
plt.plot(history.history['Recall'], label='Train Recall')  # Corrected for recall
plt.plot(history.history['val_Recall'], label='Validation Recall')  # Corrected for validation recall
plt.legend()
plt.title("Recall")
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.savefig('recall_plot_bigru.png')  # Save the plot as a PNG file
plt.close()

# Save F1 Score plot (Correct key for F1 score)
plt.figure()
plt.plot(history.history['f1_score'], label='Train F1 Score')  # Corrected for F1 score
plt.plot(history.history['val_f1_score'], label='Validation F1 Score')  # Corrected for validation F1 score
plt.legend()
plt.title("F1 Score")
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.savefig('f1_score_plot_bigru.png')  # Save the plot as a PNG file
plt.close()


In [12]:
import numpy as np
import seaborn as sns
from tensorflow.keras.preprocessing import image

In [13]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, auc

# -------- Get True Labels and Predictions --------
# Get predictions on the test set
y_pred_probs = cnn.predict(test_set1)
y_pred = np.argmax(y_pred_probs, axis=1)

# Get true labels
y_true = test_set1.classes

# -------- Confusion Matrix --------
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('confusion_matrix_bigru.png')
plt.close()

# -------- Classification Report --------
print("Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=list(test_set1.class_indices.keys())))

# -------- Multi-Class ROC-AUC --------

# Since it's multi-class, we need one-hot encode the y_true
y_true_onehot = tf.keras.utils.to_categorical(y_true, num_classes=6)

# Compute AUC for each class and macro-average
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(6):  # Number of classes
    fpr[i], tpr[i], _ = roc_curve(y_true_onehot[:, i], y_pred_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute macro-average ROC curve and AUC
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(6)]))
mean_tpr = np.zeros_like(all_fpr)
for i in range(6):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
mean_tpr /= 6
macro_roc_auc = auc(all_fpr, mean_tpr)

# -------- Plot ROC Curve --------
plt.figure(figsize=(8,6))
for i in range(6):
    plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

plt.plot(all_fpr, mean_tpr, color='navy', linestyle='--', label=f'Macro-Average (AUC = {macro_roc_auc:.2f})')
plt.plot([0,1], [0,1], color='gray', linestyle='--')
plt.title('Multi-class ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.savefig('auc_curve_bigru.png')
plt.close()


235/235 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step
Classification Report:

              precision    recall  f1-score   support

  Chickenpox       0.09      0.09      0.09       700
      Cowpox       0.08      0.08      0.08       686
        HFMD       0.21      0.21      0.21      1624
     Healthy       0.16      0.16      0.16      1162
     Measles       0.06      0.06      0.06       518
   Monkeypox       0.38      0.38      0.38      2828

    accuracy                           0.23      7518
   macro avg       0.16      0.16      0.16      7518
weighted avg       0.23      0.23      0.23      7518



In [14]:
# Save the trained model
cnn.save("monkeypox_cnn_bgru_sa.h5")

In [23]:
img_path = "C:/Users/KIIT/Music/Monkeypox/archive (60)/Augmented Images/Augmented Images/FOLDS_AUG/fold1_AUG/Train/Monkeypox/MKP_01_01_1.jpg"

In [28]:
# Load and preprocess the image
img = image.load_img(img_path, target_size=(64, 64))  # Resize to match model input size
img_array = image.img_to_array(img) / 255.0  # Normalize pixel values
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

In [29]:
# Make prediction
prediction = cnn.predict(img_array)
predicted_class = prediction[0][0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step


In [30]:
predicted_class

0.99999833

In [27]:
img_path = "C:/Users/KIIT/Music/Monkeypox/archive (60)/Augmented Images/Augmented Images/FOLDS_AUG/fold1_AUG/Train/Chickenpox/CHP_01_01_2.jpg"

In [31]:
# Set a decision threshold
threshold = 0.5  # Adjust if needed based on model performance

# Output the result
if predicted_class < threshold:
    print(f"The image {img_path} is classified as Monkeypox Positive.")
else:
    print(f"The image {img_path} is classified as Monkeypox Negative.")

The image C:/Users/KIIT/Music/Monkeypox/archive (60)/Augmented Images/Augmented Images/FOLDS_AUG/fold1_AUG/Train/Chickenpox/CHP_01_01_2.jpg is classified as Monkeypox Negative.


In [22]:
# Save the trained model
cnn.save("monkeypox_cnn_bgru_sa.keras")